<a href="https://colab.research.google.com/github/debora28/SecView/blob/debora/07_12_Crime_E2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. IMPORTS**

In [1066]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [1067]:
# Diretamente do GitHub (dataset 2014):
url = 'https://raw.githubusercontent.com/debora28/pi-2020.2/main/2014-1-10mil.csv'
data = pd.read_csv(url)

#**2.1 PREPARANDO DADOS**

In [1068]:
print(data.isnull().sum())
print(data)

Unnamed: 0                   0
NUM_BO                       0
ANO_BO                       0
ID_DELEGACIA                 0
NOME_DEPARTAMENTO            0
NOME_SECCIONAL               0
DELEGACIA                    0
NOME_DEPARTAMENTO_CIRC       0
NOME_SECCIONAL_CIRC          0
NOME_DELEGACIA_CIRC          0
ANO                          0
MES                          0
DATA_OCORRENCIA_BO          87
HORA_OCORRENCIA_BO        1040
FLAG_STATUS                  0
RUBRICA                      0
DESDOBRAMENTO             9351
CONDUTA                   4835
LATITUDE                  1593
LONGITUDE                 1593
CIDADE                       0
LOGRADOURO                 429
NUMERO_LOGRADOURO         2467
FLAG_STATUS.1                0
DESCR_TIPO_PESSOA            0
CONT_PESSOA                  0
SEXO_PESSOA                  0
IDADE_PESSOA              1902
COR                       2372
DESCR_PROFISSAO           3002
DESCR_GRAU_INSTRUCAO      2832
dtype: int64
      Unnamed: 0  NUM_BO  

In [1069]:
data.shape

(10000, 31)

Retirando colunas que não serão usadas:

In [1070]:
data = data.drop(columns=['ID_DELEGACIA','NOME_DEPARTAMENTO', 'NOME_SECCIONAL','DELEGACIA','NOME_DEPARTAMENTO_CIRC','NOME_SECCIONAL_CIRC','FLAG_STATUS','DESDOBRAMENTO','NUMERO_LOGRADOURO','FLAG_STATUS.1'])
data.to_csv('data_semDesdob.csv', index= False)
data.head()

,Unnamed: 0,NUM_BO,ANO_BO,NOME_DELEGACIA_CIRC,ANO,MES,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,DESCR_TIPO_PESSOA,CONT_PESSOA,SEXO_PESSOA,IDADE_PESSOA,COR,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
0,243,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Curador,7,F,51.0,Parda,AUXILIAR DE LIMPEZA,NaN
1,244,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Indiciado,4,M,19.0,Parda,DESEMPREGADO,1 Grau completo
2,245,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Testemunha,2,M,38.0,NaN,INVESTIGADOR(A),Superior completo
3,246,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Testemunha,6,M,23.0,Parda,DESEMPREGADO,2 Grau completo
4,270,3,2014,DEL.POL.ITAQUAQUECETUBA,2014,1,14/01/2014,17:00,Drogas,NaN,-2.348.960.369,-4.637.155.072,ITAQUAQUECETUBA,R UBATUBA,Testemunha,1,M,40.0,NaN,POLICIAL CIVIL,2 Grau completo


In [1108]:
data.isnull().sum().sum()

0

In [1109]:
data.isnull().any()

Unnamed: 0                          False
NUM_BO                              False
ANO_BO                              False
NOME_DELEGACIA_CIRC                 False
ANO                                 False
MES                                 False
DATA_OCORRENCIA_BO                  False
HORA_OCORRENCIA_BO                  False
RUBRICA                             False
CONDUTA                             False
LATITUDE                            False
LONGITUDE                           False
CIDADE                              False
LOGRADOURO                          False
DESCR_TIPO_PESSOA                   False
CONT_PESSOA                         False
SEXO_PESSOA                         False
IDADE_PESSOA                        False
COR                                 False
DESCR_PROFISSAO                     False
DESCR_GRAU_INSTRUCAO                False
Drogas                              False
Estupro                             False
Estupro de vulneravel             

Preenchendo valores NaN ou NA com os valores mais frequentes:

In [1071]:
data['CONDUTA'].value_counts()

VEICULO                         1386
TRANSEUNTE                      1369
OUTROS                           681
RESIDENCIA                       422
INTERIOR DE VEICULO              354
ESTABELECIMENTO COMERCIAL        326
CARGA                            273
INTERIOR ESTABELECIMENTO         112
ESTABELECIMENTO BANCARIO          75
ESTABELECIMENTO-OUTROS            62
INTERIOR TRANSPORTE COLETIVO      46
ESTABELECIMENTO ENSINO            37
COLETIVO                          13
CONDOMINIO RESIDENCIAL             7
CONDOMINIO COMERCIAL               2
Name: CONDUTA, dtype: int64

In [1072]:
data['CONDUTA'].fillna('VEICULO', inplace=True)

In [1073]:
data['CONDUTA'].value_counts()

VEICULO                         6221
TRANSEUNTE                      1369
OUTROS                           681
RESIDENCIA                       422
INTERIOR DE VEICULO              354
ESTABELECIMENTO COMERCIAL        326
CARGA                            273
INTERIOR ESTABELECIMENTO         112
ESTABELECIMENTO BANCARIO          75
ESTABELECIMENTO-OUTROS            62
INTERIOR TRANSPORTE COLETIVO      46
ESTABELECIMENTO ENSINO            37
COLETIVO                          13
CONDOMINIO RESIDENCIAL             7
CONDOMINIO COMERCIAL               2
Name: CONDUTA, dtype: int64

In [1074]:
data['LATITUDE'].value_counts()

-2.371.286.115    48
-2.364.027.263    23
-2.352.519.435    23
-2.347.103.556    22
-2.355.850.851    22
                  ..
-2.348.937.986     1
-2.353.336.199     1
-2.337.814.456     1
-2.353.860.314     1
-2.350.421.976     1
Name: LATITUDE, Length: 3439, dtype: int64

In [1075]:
data['LATITUDE'].fillna('-23.71286115', inplace=True)

In [1076]:
data['LONGITUDE'].value_counts()

-4.678.186.771    48
-4.681.568.811    23
-4.639.605.905    23
-4.672.128.418    22
-4.661.519.738    22
                  ..
-4.664.018.923     1
-459.909.844       1
-4.661.527.855     1
-4.654.911.801     1
-4.679.653.543     1
Name: LONGITUDE, Length: 3439, dtype: int64

In [1077]:
data['LONGITUDE'].fillna('-46.78186771', inplace=True)

In [1078]:
data['DATA_OCORRENCIA_BO'].value_counts()

01/01/2014    2556
02/01/2014    1443
03/01/2014     911
06/01/2014     550
04/01/2014     415
              ... 
25/06/2014       1
01/06/2014       1
23/05/2014       1
21/04/2014       1
03/05/2014       1
Name: DATA_OCORRENCIA_BO, Length: 155, dtype: int64

In [1079]:
data['DATA_OCORRENCIA_BO'].fillna('01/01/2014', inplace=True)

In [1080]:
data['HORA_OCORRENCIA_BO'].value_counts()

12:00    239
15:00    218
16:00    207
14:00    204
01:00    201
        ... 
12:11      1
07:44      1
11:46      1
02:39      1
19:37      1
Name: HORA_OCORRENCIA_BO, Length: 484, dtype: int64

In [1081]:
data['HORA_OCORRENCIA_BO'].fillna('12:00', inplace=True)

In [1082]:
# data['somente Horas'] = pd.to_datetime(data['HORA_OCORRENCIA_BO'], format= '%H:%M.%f') \
                        # .apply(lambda x: x.strftime('%H:%M'))

In [1083]:
data['LOGRADOURO'].value_counts()

AV SAPOPEMBA                     65
EST DO M BOI MIRIM               54
AV DO ESTADO                     39
R APA                            35
AV CELSO GARCIA                  34
                                 ..
TRAVESSA ARMANDO DA FONSECA       1
RUA ORENSE                        1
RUA ADRIANO PEREIRA               1
R DONA MARIA FERRAZ DO AMARAL     1
R ESTAMPA ESPORTIVA               1
Name: LOGRADOURO, Length: 3186, dtype: int64

In [1084]:
data['LOGRADOURO'].fillna('AV SAPOPEMBA', inplace=True)

In [1085]:
data['COR'].value_counts()

Branca     4166
Parda      2517
Preta       575
Outros      334
Amarela      36
Name: COR, dtype: int64

In [1086]:
data['COR'].fillna('Branca', inplace=True)

In [1087]:
data['IDADE_PESSOA'].value_counts()

30.0     269
26.0     260
33.0     256
28.0     245
38.0     241
        ... 
103.0      1
87.0       1
91.0       1
89.0       1
88.0       1
Name: IDADE_PESSOA, Length: 92, dtype: int64

In [1088]:
data['IDADE_PESSOA'].fillna('30', inplace=True)

In [1089]:
data['DESCR_PROFISSAO'].value_counts()

POLICIAL MILITAR    827
POLICIAL CIVIL      721
ESTUDANTE           308
MOTORISTA           300
DESEMPREGADO        281
                   ... 
ASSESSORISTA          1
AZULEJISTA            1
BORDADEIRA            1
JUIZ DE DIREITO       1
ENCADERNADOR(A)       1
Name: DESCR_PROFISSAO, Length: 417, dtype: int64

In [1090]:
data['DESCR_PROFISSAO'].fillna('POLICIAL MILITAR', inplace=True)

In [1091]:
data['DESCR_GRAU_INSTRUCAO'].value_counts()

2 Grau completo        2780
1 Grau completo        1882
Superior completo      1310
1 Grau incompleto       574
Superior incompleto     321
2 Grau incompleto       227
Analfabeto               74
Name: DESCR_GRAU_INSTRUCAO, dtype: int64

In [1092]:
data['DESCR_GRAU_INSTRUCAO'].fillna('2 Grau completo', inplace=True)

In [1093]:
print(data.isnull().any())
print()

Unnamed: 0              False
NUM_BO                  False
ANO_BO                  False
NOME_DELEGACIA_CIRC     False
ANO                     False
MES                     False
DATA_OCORRENCIA_BO      False
HORA_OCORRENCIA_BO      False
RUBRICA                 False
CONDUTA                 False
LATITUDE                False
LONGITUDE               False
CIDADE                  False
LOGRADOURO              False
DESCR_TIPO_PESSOA       False
CONT_PESSOA             False
SEXO_PESSOA             False
IDADE_PESSOA            False
COR                     False
DESCR_PROFISSAO         False
DESCR_GRAU_INSTRUCAO    False
dtype: bool



In [1094]:
data.describe()

,Unnamed: 0,NUM_BO,ANO_BO,ANO,MES,CONT_PESSOA
count,10000.00000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000
mean,4999.50000,24.377100,2014.011400,2014.0,1.362000,2.625500
std,2886.89568,16.054192,0.112567,0.0,0.898909,1.934276
min,0.00000,1.000000,2014.000000,2014.0,1.000000,1.000000
25%,2499.75000,11.000000,2014.000000,2014.0,1.000000,1.000000
50%,4999.50000,21.000000,2014.000000,2014.0,1.000000,2.000000
75%,7499.25000,37.000000,2014.000000,2014.0,1.000000,3.000000
max,9999.00000,63.000000,2017.000000,2014.0,6.000000,17.000000


#**2.2 CONVERTENDO DADOS CATEGÓRICOS EM NUMÉRICOS**
Codificação one-hot 



In [1095]:
novas_colunas = pd.get_dummies(data['RUBRICA']) 
data = pd.concat([data,novas_colunas], axis=1) 
data.head(2)

,Unnamed: 0,NUM_BO,ANO_BO,NOME_DELEGACIA_CIRC,ANO,MES,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,DESCR_TIPO_PESSOA,CONT_PESSOA,SEXO_PESSOA,IDADE_PESSOA,COR,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,Drogas,Estupro,Estupro de vulneravel,Furto,Furto qualificado,Homicidio,Homicidio culposo,Homicidio culposo por veiculo,Homicidio qualificado,Lesao corporal,Lesao corporal de natureza GRAVE,Lesao corporal seguida de morte,Lesao culposa por veiculo,Roubo
0,243,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Curador,7,F,51,Parda,AUXILIAR DE LIMPEZA,2 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,244,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Indiciado,4,M,19,Parda,DESEMPREGADO,1 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [1096]:
data['RUBRICA'].value_counts()

Roubo                               3630
Lesao corporal                      1875
Drogas                              1694
Furto                               1012
Furto qualificado                    633
Lesao culposa por veiculo            595
Homicidio                            279
Estupro de vulneravel                103
Homicidio qualificado                 96
Estupro                               50
Homicidio culposo por veiculo         22
Lesao corporal seguida de morte        4
Lesao corporal de natureza GRAVE       4
Homicidio culposo                      3
Name: RUBRICA, dtype: int64

In [1097]:
data['ROUBO_FURTO'] = data['Roubo'] + data['Furto'] + data['Furto qualificado']
data.head(2)

,Unnamed: 0,NUM_BO,ANO_BO,NOME_DELEGACIA_CIRC,ANO,MES,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,DESCR_TIPO_PESSOA,CONT_PESSOA,SEXO_PESSOA,IDADE_PESSOA,COR,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,Drogas,Estupro,Estupro de vulneravel,Furto,Furto qualificado,Homicidio,Homicidio culposo,Homicidio culposo por veiculo,Homicidio qualificado,Lesao corporal,Lesao corporal de natureza GRAVE,Lesao corporal seguida de morte,Lesao culposa por veiculo,Roubo,ROUBO_FURTO
0,243,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Curador,7,F,51,Parda,AUXILIAR DE LIMPEZA,2 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,244,2,2014,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Indiciado,4,M,19,Parda,DESEMPREGADO,1 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [1098]:
data.dtypes

Unnamed: 0                           int64
NUM_BO                               int64
ANO_BO                               int64
NOME_DELEGACIA_CIRC                 object
ANO                                  int64
MES                                  int64
DATA_OCORRENCIA_BO                  object
HORA_OCORRENCIA_BO                  object
RUBRICA                             object
CONDUTA                             object
LATITUDE                            object
LONGITUDE                           object
CIDADE                              object
LOGRADOURO                          object
DESCR_TIPO_PESSOA                   object
CONT_PESSOA                          int64
SEXO_PESSOA                         object
IDADE_PESSOA                        object
COR                                 object
DESCR_PROFISSAO                     object
DESCR_GRAU_INSTRUCAO                object
Drogas                               uint8
Estupro                              uint8
Estupro de 

In [ ]:
Conertendo a coluan 'idade' em inteiro:

In [1099]:
data.IDADE_PESSOA.astype('int64')

0       51
1       19
2       38
3       23
4       40
        ..
9995    35
9996    29
9997    41
9998    46
9999    26
Name: IDADE_PESSOA, Length: 10000, dtype: int64

In [1100]:
data.groupby(by='SEXO_PESSOA').size()

SEXO_PESSOA
F    2335
I     166
M    7499
dtype: int64

In [ ]:
Convertendo a coluna 'sexo' com valores numéricos:

In [1101]:
def codificando(coluna, codigo):
  colCodificada = pd.Series(coluna, copy=True)
  for key, value in codigo.items():
    colCodificada.replace(key, value, inplace=True)
  return colCodificada

print('Antes: ') 
print(pd.value_counts(data["SEXO_PESSOA"]))
data["SEXO_PESSOA_CODED"] = codificando(data["SEXO_PESSOA"], {'M':0,'F':1,'I':0})
print('Depois: ')
print(pd.value_counts(data["SEXO_PESSOA_CODED"]))

Antes: 
M    7499
F    2335
I     166
Name: SEXO_PESSOA, dtype: int64
Depois: 
0    7665
1    2335
Name: SEXO_PESSOA_CODED, dtype: int64


In [1102]:
# data.HORA_OCORRENCIA_BO.astype('datetime')

In [1103]:
# data = datetime.strptime(data["HORA_OCORRENCIA_BO"], "%H:%M")
# data["hour"] = data.strftime("%H:%M")

In [1104]:
# def converteHora(coluna, codigo):
#   colCodificada = pd.Series(coluna, copy=True)
#   for key, value in codigo.items():
#     datetime.strptime(times1,"%H:%M:%S.%f")
#   return colCodificada

# print('Antes: ') 
# print(pd.value_counts(data["HORA_OCORRENCIA_BO"]))
# data["HORA_OCORRENCIA_BO"] = codificando(data["HORA_OCORRENCIA_BO"], {'M':0,'F':1,'I':0})
# print('Depois: ')
# print(pd.value_counts(data["HORA_OCORRENCIA_BO"]))

#**2.3 SEPARAR A VARIÁVEL ALVO**

In [1105]:
x = data[['HORA_OCORRENCIA_BO','IDADE_PESSOA']].copy()
y = data[['ROUBO_FURTO']].copy()

#**3. DIVIDIR TESTE E TREINAMENTO**

In [1106]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=500)

#**4. CRIAR MODELO DE ML**

In [1107]:
tipos_crimes_classifier = DecisionTreeClassifier(max_leaf_nodes=100, random_state=1)
tipos_crimes_classifier.fit(x_train, y_train)

ValueError: ignored

In [ ]:
print(tipos_crimes_classifier)

#**5. FAZER PREDIÇÃO**

In [ ]:
predictions = tipos_crimes_classifier.predict(x_test)

#**6. TESTAR ACURÁCIA**

In [ ]:
accuracy_score(y_true=y_test, y_pred=predictions)